In [1]:
!pip install --upgrade -q gspread
!pip install -U -q PyDrive


import gspread
import pandas as pd
from google.colab import auth
auth.authenticate_user()
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
from google.colab import drive as gdrive

gdrive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
y_test_fname = '/content/gdrive/MyDrive/ML práctico 2022/2022-2C/y_test.jl'

with open(y_test_fname) as f:
  y_test = list(map(float, f.read().split('\n')))

In [9]:
import numpy as np
from collections import Counter
from time import sleep
from sklearn.metrics import mean_squared_error, mean_absolute_error, \
      mean_absolute_percentage_error
from IPython.display import clear_output, display
import pytz

tz = pytz.timezone('America/Buenos_Aires')

class Evaluator:
  _cached_scores = {}

  def get_submissions(self):
    worksheet = gc.open('Taller de model selection (Responses)')
    rows = worksheet.worksheets()[0].get_all_values()
    header = rows[0]
    rows = [dict(zip(header, row)) for row in rows[1:]]
    return rows
    
  def get_competition_table(self):
    rows = self.get_submissions()

    team_submissions = Counter()
    team_best = {}

    for row in rows:
      team_name = row['Nombre equipo']
      team_submissions[team_name] += 1
      
      id = row['Submission'].split('=')[-1]
      result = self.score_solution(id)
      if result is None: continue

      result.update(row)

      if team_name not in team_best:
        team_best[team_name] = result
      elif result['log(rmse)'] < team_best[team_name]['log(rmse)']:
        team_best[team_name] = result

    docs = []
    for team_name, best_result in team_best.items():
      doc = {}
      doc.update(best_result)
      doc['team_name'] = team_name
      doc['submissions'] = team_submissions[team_name]
      docs.append(doc)

    df = pd.DataFrame(docs).sort_values('log(rmse)')

    now = datetime.now(tz=tz)

    secs_ago = lambda t: pd.Timedelta(
        now - tz.localize(datetime.strptime(t, '%m/%d/%Y %H:%M:%S'))
        ).round('S')

    df['secs ago'] = df.Timestamp.apply(secs_ago)
    return df[['team_name', 'log(rmse)', 'secs ago', 'submissions', 
              'log(mae)', 'mape', 'Comment']]

  def score_solution(self, id):
      if id not in self._cached_scores:
        downloaded = drive.CreateFile({'id':id})   # replace the id with id of file you want to access
        downloaded.GetContentFile('tmp_submission.csv') 
        with open('tmp_submission.csv') as f:
          y_pred = list(map(float, f.read().split()))
          
        if len(y_pred) != y_test: 
          # v1 data submission
          self._cached_scores[id] = None
        else:
          self._cached_scores[id] = {
              'log(rmse)': np.log10(mean_squared_error(y_test, y_pred) ** 0.5),
              'log(mae)': np.log10(mean_absolute_error(y_test, y_pred)),
              'mape': mean_absolute_percentage_error(y_test, y_pred),
          }

      return self._cached_scores[id]


In [10]:
ev = Evaluator()

In [11]:
from datetime import datetime
from collections import Counter
from time import sleep, time
from sklearn.metrics import mean_squared_error
from IPython.display import clear_output, display, display_markdown

pd.options.display.float_format = lambda x :f'{x:.02f}'

df = None
last_print = 0
while True:
  prev_df = df
  df = ev.get_competition_table()
  changed = (
      prev_df is None or 
      len(df) != len(prev_df) or 
      (df['log(rmse)'] != prev_df['log(rmse)']).any()
  )
  if changed or time() - last_print > 60: 
    clear_output()
    display_markdown(f'**Hora**: {datetime.now(tz).strftime("%H:%M:%S")}', raw=True)
    display(df)
    last_print = time()
  sleep(10)

KeyError: ignored